# 環境設置

## 載入套件
- selenium、pandas、requests、bs4、time

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  # 設定 driver 的行為
from selenium.webdriver.support.ui import Select  # 選擇＂下拉式選單＂
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # 載入常見錯誤
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  # 更改載入策略
from selenium.webdriver.support.ui import WebDriverWait  # 等待機制
from selenium.webdriver.support import expected_conditions as EC  # 預期事件
from selenium.webdriver.common.by import By  # 找尋元素的方法
import pandas as pd  # 載入 pandas
import pandas.io.formats.excel  # 輸出自定義格式 Excel
import requests
import requests.packages.urllib3

requests.packages.urllib3.disable_warnings()  # 關閉錯誤警告
from urllib.request import HTTPError  # 載入 HTTPError
from bs4 import BeautifulSoup
import time  # 強制等待

## 設定 driver 的參數：options、desired_capabilities

In [2]:
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--start-maximized")  # 視窗最大化
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
my_capabilities = DesiredCapabilities.CHROME
my_capabilities[
    "pageLoadStrategy"] = "none"  # 當 html下載完成之後，不等待解析完成，selenium會直接返回

# 輸出用的 function

## organize_columns(df1)
- 處理欄位：圖書館、館藏地（c2）、索書號（c3）、館藏狀態（c4）、連結

In [37]:
def organize_columns(df1):
    # 處理 column 2：館藏地
    c2 = ['館藏地/室', '館藏室', '館藏地/館藏室', '館藏地', '典藏館', '館藏位置', '館藏地/區域', '典藏地名稱']
    df1['c2'] = ''
    for c in c2:
        try:
            df1['c2'] += df1[c]
        except:
            pass

    # 處理 column 3：索書號
    c3 = ['索書號', '索書號/期刊合訂本卷期', '索書號 / 部冊號']
    df1['c3'] = ''
    for c in c3:
        try:
            df1['c3'] += df1[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = [
        '狀態/到期日', '目前狀態 / 到期日', '館藏狀態', '處理狀態', '狀態 (說明)', '館藏現況 說明',
        '目前狀態/預計歸還日期', '圖書狀況 / 到期日'
    ]
    df1['c4'] = ''
    for c in c4:
        try:
            df1['c4'] += df1[c]
        except:
            pass

    # 直接生成另一個 DataFrame
    df2 = pd.DataFrame()
    df2['圖書館'] = df1['圖書館']
    df2['館藏地'] = df1['c2']
    df2['索書號'] = df1['c3']
    df2['館藏狀態'] = df1['c4']
    df2['連結'] = df1['連結']

    return df2

## set_excel(df, directory)
- 待

In [4]:
def set_excel(df, directory):
    # B｢圖書館｣、C「館藏地」、D「索書號」、E「館藏狀態」、F「連結」
    pandas.io.formats.excel.header_style = None  # 標題格式清除
    writer = pd.ExcelWriter(directory)
    df.to_excel(writer, sheet_name="搜尋結果")

    workbook1 = writer.book
    worksheets = writer.sheets
    worksheet1 = worksheets["搜尋結果"]

    # 測試
    cell_format = workbook1.add_format({
        "font_name": "微軟正黑體",
        "font_size": 16,
        "align": "left",
        #         "border": 80,
    })
    worksheet1.set_column("B:F", 40, cell_format)

    # 設定單元格的寬度
    #     worksheet1.set_column("B:F", 35)

    writer.save()
    print("爬取完成")

# 爬蟲程式

## webpac_gov_crawler(org, org_url, ISBN)
- 『函式完成度』：
- 『運作的原理』：宜蘭縣公共圖書館、桃園市立圖書館、高雄市立圖書館、屏東縣公共圖書館、花蓮縣圖書館、澎湖縣圖書館
- 『適用的機構』：
- 『能處理狀況』：判斷搜尋結果有沒有超過一筆、只有一筆搜尋結果有沒有跳轉、找不到書
- 『下一步優化』：當搜尋無結果時，可以直接結束。

### click_more_btn()

In [5]:
def click_more_btn(driver):
    try:
        while True:
            more_btn = wait.until(
                EC.visibility_of_element_located(
                    (By.CSS_SELECTOR, "a.btnstyle.bluebg3.morewidth")))
            if more_btn:
                time.sleep(2.5)  # 不得已的強制等待
                more_btn.click()
                # driver.find_element_by_css_selector("a.btnstyle.bluebg3.morewidth").click()
    except:
        time.sleep(2.5)

### webpac_gov_crawler

In [ ]:
def webpac_gov_crawler(org, org_url, ISBN):
    try:
        # 進入＂進階搜尋首頁＂
        search_url = url + ISBN
        driver.get(org_url)

        # 計算＂搜尋結果的數量＂
        book_list = wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.booklist")))
        results = len(book_list)
        print(f"搜尋結果數量有：{results}筆", end="    ")

        time.sleep(2)  # 頁面加載，用強制等待

        # 依照搜尋結果的數量分成：一筆和多筆兩種情形
        if results < 2:
            if wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "div.bookplace_list"))):
                click_more_btn(driver)
                return pd.read_html(driver.page_source, encoding="utf-8")[-1]
        else:
            soup = BeautifulSoup(driver.page_source,
                                 "html.parser")  # 用以當搜尋結果多筆時，找尋不同＂詳細書目＂的網址

            tgt = []
            url = url.split("search?")[0]  # 取得＂搜尋主頁＂頁面
            for li in soup.find_all("div", "booklist"):
                href = url + li.find("h2").find("a")["href"]

                driver.get(href)
                if wait.until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, "div.bookplace_list"))):
                    click_more_btn(driver)
                tgt.append(
                    pd.read_html(driver.page_source, encoding="utf-8")[-1])
            return pd.concat(tgt, axis=0, ignore_index=True)
    except:
        print(f"「{url}」無法爬取！")

### TEST ZONE for webpac_gov_crawler()

In [ ]:
webpac_gov_crawler(
    org='宜蘭縣公共圖書館',
    org_url='https://webpac.ilccb.gov.tw/',
    ISBN='9789869109321'
)

## webpac_jsp_crawler(org, org_url, ISBN)
- 『函式完成度』：高
- 『運作的原理』：
- 『適用的機構』：國立宜蘭大學、佛光大學、嘉南藥理大學
- 『能處理狀況』：
- 『下一步優化』：

In [ ]:
def webpac_jsp_crawler(org, url, ISBN):
    try:
        # url = "http://libils.fgu.edu.tw/webpacIndex.jsp"
        # ISBN = "9789573317241"  # 哈利波特(1)：神秘的魔法石

        test_url = url[:]  # 測試用，輸出「現在測試的網頁」
        test_start = time.time()  # 測試用，計算程式進行時間

        # 初始化 driver；設定 explicit waits
        driver = webdriver.Chrome(options=my_options,
                                  desired_capabilities=my_capabilities)
        wait = WebDriverWait(driver, 30)  # 最長等待時間：30 秒；查找元素的間隔時間：0.5秒（default）

        # 進入＂搜尋主頁＂
        driver.get(url)

        # 等待定位＂下拉式選單＂，選擇 ISBN
        search_field = wait.until(
            EC.presence_of_element_located((By.NAME, "search_field")))
        select = Select(search_field)
        select.select_by_value("ISBN")

        org = driver.title

        # 等待定位＂搜尋欄＂，輸入 ISBN
        search_input = wait.until(
            EC.presence_of_element_located((By.NAME, "search_input")))
        search_input.send_keys(ISBN)
        search_input.submit()

        # 依據有沒有＂div.mainCon＂，來判斷搜尋結果是「＂一筆＂」還是「＂零筆＂或＂多筆＂」
        try:  # 3 秒內可以找到元素，表示只有＂一筆＂搜尋結果
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.mainCon")))

            # 依據有沒有＂table.order＂，來判斷該頁面＂有＂表格還是＂沒有＂表格
            # 例如：「http://192.192.231.232/bookDetail.do?id=260965&resid=188809782&nowid=3」，
            # 有頁面但沒有表格
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "table.order")))
                tgt = pd.read_html(driver.page_source,
                                   encoding="utf-8")[-2]  # 定位＂表格＂，爬取內容
                tgt["圖書館"], tgt["連結"] = org, driver.current_url

                driver.close()
                test_end = time.time()  # 測試用，計算程式進行時間
                print(f"程式執行成功，執行時間：{test_end - test_start:7.4f}秒，在「{url}」"
                      )  # 測試用，計算程式進行時間
                return tgt.fillna("")
            except:  # 5 秒內＂找不到＂ table.order，表示該頁面有表格
                print(f"該頁面沒有表格：「{driver.current_url}」")
        except (NoSuchElementException,
                TimeoutException):  # 3 秒內＂找不到＂元素，，表示有＂多筆＂搜尋結果
            root_url = url.replace(url.split("/")[-1],
                                   "")  # root_url 用來組成不同的＂詳細書目＂的網址

            tgt = []
            # 定位＂內嵌框架＂，切換至此內部，找尋多個＂詳細書目的網址＂
            iframe = wait.until(
                EC.presence_of_element_located((By.ID, "leftFrame")))
            driver.switch_to.frame(iframe)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # 組合網址
            tgt_urls = []
            for anchor in soup.find_all("a", "bookname"):
                tgt_urls.append(root_url + anchor["href"])

            # 進入不同的＂詳細書目＂
            for urlx in tgt_urls:
                driver.get(urlx)

                # 依據有沒有＂table.order＂，來判斷該頁面＂有＂表格還是＂沒有＂表格
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, "table.order")))
                    sec_tgt = pd.read_html(driver.page_source,
                                           encoding="utf-8")[-2]  # 定位＂表格＂，爬取內容
                    sec_tgt["圖書館"], sec_tgt["連結"] = org, driver.current_url
                    tgt.append(sec_tgt)
                except:
                    print(f"該頁面沒有表格：「{driver.current_url}」")

            driver.close()
            if tgt != []:
                tgt = pd.concat(tgt, axis=0, ignore_index=True)
                test_end = time.time()  # 測試用，計算程式進行時間
                print(f"程式執行成功，執行時間：{test_end - test_start:7.4f}秒，在「{url}」"
                      )  # 測試用，計算程式進行時間
                return tgt.fillna("")
        except Exception as e:
            test_end = time.time()  # 測試用，計算程式進行時間
            print("=" * 100)
            print(f"程式執行失敗，執行時間：{test_end - test_start:7.4f}秒，在「{org}」")
            print(f"網址為：「{url}」")
            print(f"錯誤訊息為：「{e}」")
            print("=" * 100)
    except:
        pass

## easy_crawler(table_position, org, org_url, ISBN)
- 『函式完成度』：高
- 『運作的原理』：直接進入＂詳細書目＂頁面後，使用 pd.read_html 抓取 table，但每個機構的 table 位置不同，需要在 function 將此當作一個 parameter。
- 『適用的機構』：國立臺灣海洋大學、國立臺灣師範大學、中央研究院、國立陽明大學、國立中央大學、中原大學、國立臺灣科技大學、中國文化大學、輔仁大學
- 『能處理狀況』：HTTPError
- 『下一步優化』：師大查詢結果有多本（https://opac.lib.ntnu.edu.tw/search*cht/i?SEARCH=9789573229131 ）

In [41]:
def easy_crawler(table_position, org, org_url, ISBN):
    try:
        # 組合成書本的網址
        tgt_url = org_url + ISBN
        # 載入 html，如果發生 HTTPError，那麼就使用 requests.get(url, verify=False)
        try:
            tgt = pd.read_html(tgt_url, encoding="utf-8")
        except HTTPError:
            resp = requests.get(tgt_url,
                                verify=False)  # 設定 verify=False，以解決 SSLError
            tgt = pd.read_html(resp.text, encoding="utf-8")
        # 定位表格
        table = tgt[table_position]
        table['圖書館'], table['連結'] = org, tgt_url
        table = organize_columns(table)
        return table  # 完成抓取 table
    except:
        print(f'《{ISBN}》在「{url}」無法爬取')

### TEST ZONE for easy_crawler()

In [42]:
if __name__ == '__main__':
    table = easy_crawler(
        table_position=4,
        org='國立臺灣師範大學',
        org_url='https://opac.lib.ntnu.edu.tw/search*cht/i?SEARCH=',
        ISBN='9789869109321'
    )
table

,圖書館,館藏地,索書號,館藏狀態,連結
0,國立臺灣師範大學,總館5F書架,330.18 081,到期 11-20-21,https://opac.lib.ntnu.edu.tw/search*cht/i?SEAR...
1,國立臺灣師範大學,總館5F書架,330.18 081,可外借,https://opac.lib.ntnu.edu.tw/search*cht/i?SEAR...
2,國立臺灣師範大學,公館分館,330.18 081 2015,到期 09-16-21,https://opac.lib.ntnu.edu.tw/search*cht/i?SEAR...


## webpac_ajax_page_crawler(org, org_url, ISBN)
- 『函式完成度』：極高
- 『運作的原理』：使用 selenium 進行搜索，進入＂詳細書目＂頁面後，從該網址分析並得到 mid，在由此進入 ajax_page。
- 『適用的機構』：新北市立圖書館、高雄市立空中大學、國立屏東大學
- 『能處理狀況』：判斷搜尋結果有沒有超過一筆、只有一筆搜尋結果有沒有跳轉、找不到書
- 『下一步優化』：當搜尋無結果時，可以直接結束。

In [ ]:
def webpac_ajax_page_crawler(org, org_url, ISBN):
    try:
        # 進入＂搜尋主頁＂
        driver.get(org_url)
        # 等待點擊＂進階查詢＂按鈕，接著點擊
        WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.LINK_TEXT, '進階查詢'))).click()
        # 等待定位＂下拉式選單＂，選擇以 ISBN 方式搜尋
        search_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'as_type_1')))
        select = Select(search_field)
        select.select_by_value('i')
        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = driver.find_element_by_id('as_keyword_1')
        search_input.send_keys(ISBN)
        search_input.send_keys(
            Keys.ENTER)  # 無法 submit()，用 send_keys(keys.ENTER) 來替代

        # 在＂搜尋結果頁面＂，等待定位＂詳細書目＂。
        # try-except 來判斷有沒有在＂搜尋結果頁面＂
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.LINK_TEXT, '詳細書目')))
        except:
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, 'div.book-detail')))

                # 抓取方式：找出 mid 後，進入 ajax pag 抓取 DataFrame
                org_url = org_url.replace('/search.cfm', '')
                tgts = []
                url = driver.current_url
                mid = url.split('mid=')[-1].split('&')[0]
                ajax_page_url = f'{org_url}/ajax_page/get_content_area.cfm?mid={mid}&i_list_number=250&i_page=1&i_sory_by=1'
                tgt = pd.read_html(ajax_page_url, encoding='utf-8')[0]
                tgt['圖書館'], tgt['連結'] = org, url
                tgts.append(tgt)
                table = pd.concat(tgts, axis=0, ignore_index=True)
                table = organize_columns(table)
                return table  # 完成抓取 table
            except:  # 沒有搜尋結果，也沒有進入＂詳細書目頁面＂
                print(f'《{ISBN}》查無此書')
                return  # 什麼都不做，退出此 function

        # 抓取多個＂詳細書目＂的網址
        anchors = driver.find_elements_by_link_text('詳細書目')
        urls = []
        for anchor in anchors:
            urls.append(anchor.get_attribute('href'))

        # 抓取方式：找出 mid 後，進入 ajax pag 抓取 DataFrame
        org_url = org_url.replace('/search.cfm', '')
        tgts = []
        for url in urls:
            mid = url.split('mid=')[-1].split('&')[0]  # 抓取 mid
            ajax_page_url = f'{org_url}/ajax_page/get_content_area.cfm?mid={mid}&i_list_number=250&i_page=1&i_sory_by=1'
            tgt = pd.read_html(ajax_page_url, encoding='utf-8')[0]
            tgt['圖書館'], tgt['連結'] = org, url
            tgts.append(tgt)
        table = pd.concat(tgts, axis=0, ignore_index=True)
        table = organize_columns(table)
        return table  # 完成抓取 table
    except:
        print(f'《{ISBN}》在「{url}」無法爬取')

### TEST ZONE for webpac_ajax_page_crawler()

In [ ]:
driver = webdriver.Chrome(options=my_options,
                          desired_capabilities=my_capabilities)

In [ ]:
table = webpac_ajax_page_crawler(
    org='新北市立圖書館',
    org_url='https://webpac.tphcc.gov.tw/webpac/search.cfm',
    ISBN='9789869109321')
table

In [ ]:
table = webpac_ajax_page_crawler(
    org='高雄市立空中大學',
    org_url='https://webpac.ouk.edu.tw/webpac/search.cfm',
    ISBN='9789869109321')
table

In [ ]:
table = webpac_ajax_page_crawler(
    org='國立屏東大學',
    org_url='https://webpac.nptu.edu.tw/webpac/search.cfm',
    ISBN='9789869109321')
table

## klccab_crawler(org, ISBN, driver, wait)
- 『函式完成度』：
- 『運作的原理』：
- 『適用的機構』：
- 『能處理狀況』：
- 『下一步優化』：

In [ ]:
def klccab_crawler(org, ISBN, driver, wait):
    try:
        url = f"https://webpac.klccab.gov.tw/webpac/search.cfm?m=as&k0={ISBN}&t0=i&c0=and&y10=&y20=&cat0=&dt0=&l0=&lv0=&lc0="
        driver.get(url)

        time.sleep(5)  # 基隆的系統太詭異了，強制等待
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = len(soup.find_all("div", "list_box"))
        if results < 2:
            wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "table.list.list_border")))
            tgt = pd.read_html(driver.page_source)[0]
        else:
            tgt = []
            for li in soup.find_all("div", "list_box"):
                url_temp = "https://webpac.klccab.gov.tw/webpac/" + li.find(
                    "a", "btn")["href"]
                driver.get(url_temp)
                wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "table.list.list_border")))
                tgt.append(
                    pd.read_html(driver.page_source, encoding="utf-8")[0])
            tgt = pd.concat(tgt, axis=0, ignore_index=True)
        return tgt
    except:
        print(f"「{url}」無法爬取！")

## tpml_crawler(org, url, ISBN, driver, wait)
- 『函式完成度』：
- 『運作的原理』：
- 『適用的機構』：
- 『能處理狀況』：
- 『下一步優化』：

In [ ]:
def tpml_crawler(org, url, ISBN, driver, wait):
    # 進入《館藏系統》頁面
    driver.get(url)

    # 等待＜下拉式選單＞元素出現
    wait.until(EC.presence_of_element_located((By.NAME, "search_field")))

    # 定位＜下拉式選單＞，以 ISBN 搜尋方式搜尋
    select = Select(driver.find_element_by_name("search_field"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("search_input")
    search_input.send_keys(ISBN)
    search_input.submit()

    # 等待＜表格＞出現
    wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "table.order")))

    # 取得當前網頁的 html 進行解析，以取得 DataFrame
    html_text = driver.page_source

    dfs = pd.read_html(html_text, encoding="utf-8")
    tgt = dfs[19]
    return tgt

## ntu_crawler(ISBN, driver)
- 『函式完成度』：
- 『運作的原理』：
- 『適用的機構』：
- 『能處理狀況』：
- 『下一步優化』：

In [ ]:
def ntu_crawler(ISBN, driver):
    df_lst = []

    url = "https://ntu.primo.exlibrisgroup.com/discovery/search?query=any,contains," + ISBN + "&tab=Everything&search_scope=MyInst_and_CI&vid=886NTU_INST:886NTU_INST&offset=0"
    driver.get(url)
    time.sleep(5)
    title = driver.find_element_by_class_name('item-title').click()

    time.sleep(3)
    try:
        try:
            back = driver.find_element_by_css_selector(
                ".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple"
            )
        except:
            back = None
        if back != None:
            back.click()

        thelist = driver.find_elements_by_class_name(
            "layout-align-space-between-center.layout-row.flex-100")
        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    new_row = [
                        '台灣大學', where[_].text, a[4].text, a[0].text, url
                    ]
                    df_lst.append(new_row)
                    break
                break
    except:
        return pd.DataFrame(df_lst)

## 國家圖書館

In [ ]:
def 國家圖書館(url, ISBN, driver):
    try:
        # 進入＂搜尋主頁＂
        driver.get(url)

        # 定位＂下拉式選單＂，選擇 ISBN
        select = Select(driver.find_element_by_name("find_code"))
        select.select_by_value("ISBN")
        search_input = driver.find_element_by_name("request")
        search_input.send_keys(ISBN)
        search_input.submit()

        # 點擊＂書在哪裡(請點選)＂
        driver.find_element_by_xpath(
            '/html/body/table[10]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/a'
        ).click()

        # 定位＂表格＂，爬取內容
        tgt = pd.read_html(driver.page_source)[-2]  # 會有 NaN，要加 .fillna("")？

        # 清理、整理資料
        tgt['連結'], tgt['圖書館'] = driver.current_url, '國家圖書館'
        tgt = tgt[['圖書館', '分館/專室', '索書號', '狀態/調閱日期', '連結']]  # 篩選 column label
        tgt.set_axis(['圖書館', '館藏地', '索書號', '館藏狀態', '連結'],
                     axis='columns',
                     inplace=True)  # 重新命名 column label
        return tgt
    except:
        print(f"「{url}」無法爬取！")